# EE 379K Lab 3
## Rohan Nagar and Wenyang Fu

In [2]:
import numpy as np

# Question 1: Jaccard Similarity

In [3]:
# def jaccard_similarity(x, y, z):
#     intersection_cardinality = len(set.intersection(*[set(x), set(y), set(z)]))
#     union_cardinality = len(set.union(*[set(x), set(y), set(z)]))
#     return intersection_cardinality / float(union_cardinality)

# jaccard_similarity(['nike', 'running', 'shoe'],
#                    ['nike', 'black', 'running', 'shoe'],
#                    ['nike', 'blue', 'jacket', 'adidas'])

def jaccard_similarity(x, y, z):
    intersection_cardinality = len(x & y & z)
    union_cardinality = len(x | y | z)
    return intersection_cardinality / float(union_cardinality)

jaccard_similarity({'nike', 'running', 'shoe'},
                   {'nike', 'black', 'running', 'shoe'},
                   {'nike', 'blue', 'jacket', 'adidas'})

0.14285714285714285

# Question 2: Minhash

### Part A: Create the characteristic matrix with the alphabet as the seven words {'nike', 'running', 'shoe', 'black', 'blue', 'jacket', 'adidas'}

In [21]:
char_matrix = np.array([['nike',1,1,1],
                         ['running',1,1,0],
                         ['shoe',1,1,0],
                         ['black',0,1,0],
                         ['blue',0,0,1],
                         ['jacket',0,0,1],
                         ['adidas',0,0,1]])
print(char_matrix)

[['nike' '1' '1' '1']
 ['running' '1' '1' '0']
 ['shoe' '1' '1' '0']
 ['black' '0' '1' '0']
 ['blue' '0' '0' '1']
 ['jacket' '0' '0' '1']
 ['adidas' '0' '0' '1']]


### Part B: For a random permutation of the seven alphabet elements, find a way to compute the first non-zero element of each column (each set) under the permutation.

In [22]:
def first_elements(permutation):
    first = []
    for column in permutation.T:
        for i, elem in enumerate(column):
            if elem == '1':
                first.append(permutation.T[0][i])
                break
    return first
    
permutation = np.random.permutation(char_matrix)
first_elements(permutation)

['running', 'running', 'jacket']

### Part C: Instead of choosing a random permutation, use the hash function $h(x) = 3x + 2 (mod 7)$.

In [23]:
def hash_func(x):
    return (3 * x + 2) % 7

hashf = np.vectorize(hash_func) # Vectorized function for hardware-parallel computation

def hashed_permutation(char_matrix):
    num_sets = char_matrix.shape[0]
    idxs = hashf(np.arange(num_sets))
    print(idxs)
    return char_matrix[idxs]

hashed_matrix = hashed_permutation(char_matrix)
print(hashed_matrix)
print(type(hashed_matrix))

first_elements(hashed_matrix)

[2 5 1 4 0 3 6]
[['shoe' '1' '1' '0']
 ['jacket' '0' '0' '1']
 ['running' '1' '1' '0']
 ['blue' '0' '0' '1']
 ['nike' '1' '1' '1']
 ['black' '0' '1' '0']
 ['adidas' '0' '0' '1']]
<class 'numpy.ndarray'>


['shoe', 'shoe', 'jacket']

### Part D: Generate your own hash functions of the form $h(x) = ax + b (mod 7)$ by choosing $a$ and $b$ at random from the set $\{0, 1, ..., 6\}$. Do this twenty times to estimate the Jaccard Similarity of the three sets. How closely do you approximate the true values, computed in question one?

In [85]:
choices = np.arange(7)
s1, s2, s3 = (  {'nike', 'running', 'shoe'},
                {'nike', 'black', 'running', 'shoe'},
                {'nike', 'blue', 'jacket', 'adidas'}
             )
alphabet = ['nike', 'running', 'shoe', 'black', 'blue', 'jacket', 'adidas']

def gen_hash_func(choices):
    a = np.random.choice(choices)
    b = np.random.choice(choices)

    hash_func = lambda x: (a * x + b) % 7
    return hash_func

hash_funcs = [gen_hash_func(choices) for _ in range(20)]
alphabet_dict = dict(zip(alphabet, range(len(alphabet)))) # Integer mapping of the alphabet.

s1 = {alphabet_dict[elem] for elem in s1}
s2 = {alphabet_dict[elem] for elem in s2}
s3 = {alphabet_dict[elem] for elem in s3}

sets = (s1, s2, s3) # Tuple of 3 sets

def jac_sim_experiment(sets, hashf):
    hashed = [{hashf(e) for e in s}
                        for s in sets]
    
    jaccard_score = jaccard_similarity(hashed[0], hashed[1], hashed[2])
    return jaccard_score

experimental_avg = sum([jac_sim_experiment(sets, f) for f in hash_funcs]) / 20
print(experimental_avg)


0.22857142857142856


### Running the experiment a few times, the experimental score ranges anywhere from perfect (1/7) to around .48. So its usually not too accurate.

# Question 3: Implementing Minhash

### Repeat the above where instead of permuting the entire characteristic matrix, implement the algorithm described in Chapter 3 of MMDS for implementing Minhash.

In [128]:
def minhash(a, hash_funcs):
    """ This implementation of MinHash
    will take a 2-dimensional characteristic matrix 
    and return the hashed signature matrix.
    
    a - 2-dim ndarray 
    hash_funcs - set of hash functions to use
    """
    alphabet_size = a.shape[0]
    num_sets = a.shape[1]
    n = len(hash_funcs)
    
    sig = np.zeros((n, num_sets)) # preallocate Signature matrix
    sig.fill(np.inf)
    
    for r in range(alphabet_size):
        row_hashes = np.array([hash_funcs[i](r) for i in range(n)])
        col_idxs = np.where(a[r, :] == '1')[0] # Columns to fill in
        num_idxs = len(col_idxs)
        sig[:, col_idxs] = np.minimum(sig[:, col_idxs], 
                                      np.array([row_hashes] * num_idxs).T)
    return sig

char_matrix = np.array([['nike',1,1,1],
                         ['running',1,1,0],
                         ['shoe',1,1,0],
                         ['black',0,1,0],
                         ['blue',0,0,1],
                         ['jacket',0,0,1],
                         ['adidas',0,0,1]])
sig = minhash(char_matrix[:, 1:] ,hash_funcs[:5])

sig

array([[ 1.,  1.,  0.],
       [ 0.,  0.,  1.],
       [ 2.,  2.,  0.],
       [ 2.,  2.,  0.],
       [ 0.,  0.,  1.]])

In [133]:
num_rows, num_cols = sig.shape

sets = [set(sig[i, :]) for i in range(num_cols)]
jaccard_score = len(set.intersection(*sets)) / len(set.union(*sets))
jaccard_score

0.3333333333333333

# Question 4: More Minhash (Shingling)

### Part A: Load the 5 article excerpts in $\tt{Lab3articles-5.txt}$.

In [37]:
documents = []
with open('Lab3articles-5.txt', 'r') as f:
    for line in f:
        documents.append(line.strip())
    
print(documents[0])

t120 The Supreme Court in Johnnesberg on Friday postponed until March 14 a hearing on a petition by government minister Winnie Mandela to prevent police reading documents seized from her home, the SAPA news agency reported. David Wright and Carlos Delgado homered and Jorge Sosa won for the sixth time as the New York Mets snapped a four-game losing streak with a 3-0 victory over Detroit on Friday night. US Defense Secretary Robert Gates said on Sunday that Iran was not yet able to make a nuclear weapon and that its program was progressing slower than Tehran expected. A Palestinian suicide bomber blew himself up in a crowded hotel dining room here Wednesday evening just as more than 200 people gathered for their Passover holiday meal, killing at least 19 and wounding more than 100 others, many of them children. OPEC kingpin Saudi Arabia signalled Tuesday it could act alone to meet a predicted increase in demand for oil, as it pushed hesitant fellow members of the cartel to raise producti

### Part B: Use $\tt{stopwords}$ from $\tt{nltk.corpus}$ to strip the stopwords from the 5 articles.

In [40]:
from nltk.corpus import stopwords

stop_words = stopwords.words('english')

stripped_documents = []
for document in documents:
    excerpt = [word for word in document.split() if word not in stop_words]
    stripped_documents.append(excerpt)
    
print(stripped_documents[0])

['t120', 'The', 'Supreme', 'Court', 'Johnnesberg', 'Friday', 'postponed', 'March', '14', 'hearing', 'petition', 'government', 'minister', 'Winnie', 'Mandela', 'prevent', 'police', 'reading', 'documents', 'seized', 'home,', 'SAPA', 'news', 'agency', 'reported.', 'David', 'Wright', 'Carlos', 'Delgado', 'homered', 'Jorge', 'Sosa', 'sixth', 'time', 'New', 'York', 'Mets', 'snapped', 'four-game', 'losing', 'streak', '3-0', 'victory', 'Detroit', 'Friday', 'night.', 'US', 'Defense', 'Secretary', 'Robert', 'Gates', 'said', 'Sunday', 'Iran', 'yet', 'able', 'make', 'nuclear', 'weapon', 'program', 'progressing', 'slower', 'Tehran', 'expected.', 'A', 'Palestinian', 'suicide', 'bomber', 'blew', 'crowded', 'hotel', 'dining', 'room', 'Wednesday', 'evening', '200', 'people', 'gathered', 'Passover', 'holiday', 'meal,', 'killing', 'least', '19', 'wounding', '100', 'others,', 'many', 'children.', 'OPEC', 'kingpin', 'Saudi', 'Arabia', 'signalled', 'Tuesday', 'could', 'act', 'alone', 'meet', 'predicted', 'i

### Part C: Compute the $k$-shingles of the documents for $k = 2$, where you shingle on the words, not letters.

In [68]:
def k_shingles_words(documents, k=2):
    all_shingles = []
    for document in documents:
        shingles = []
        size = len(document)
        for i, word in enumerate(document):
            if i >= size - k: break
            for j in range(i+1, i+k):
                word += (' ' + document[j])
            shingles.append(word)
            
        all_shingles.append(shingles)
        
    return all_shingles
        
print(k_shingles_words(stripped_documents)[0])

['t120 The', 'The Supreme', 'Supreme Court', 'Court Johnnesberg', 'Johnnesberg Friday', 'Friday postponed', 'postponed March', 'March 14', '14 hearing', 'hearing petition', 'petition government', 'government minister', 'minister Winnie', 'Winnie Mandela', 'Mandela prevent', 'prevent police', 'police reading', 'reading documents', 'documents seized', 'seized home,', 'home, SAPA', 'SAPA news', 'news agency', 'agency reported.', 'reported. David', 'David Wright', 'Wright Carlos', 'Carlos Delgado', 'Delgado homered', 'homered Jorge', 'Jorge Sosa', 'Sosa sixth', 'sixth time', 'time New', 'New York', 'York Mets', 'Mets snapped', 'snapped four-game', 'four-game losing', 'losing streak', 'streak 3-0', '3-0 victory', 'victory Detroit', 'Detroit Friday', 'Friday night.', 'night. US', 'US Defense', 'Defense Secretary', 'Secretary Robert', 'Robert Gates', 'Gates said', 'said Sunday', 'Sunday Iran', 'Iran yet', 'yet able', 'able make', 'make nuclear', 'nuclear weapon', 'weapon program', 'program pr

### Part D: Compute the $k$-shingles of the documents for $k = 3$, where you shingle on the characters, not words.

In [85]:
def k_shingles_characters(documents, k=3):
    all_shingles = []
    for doc in documents:
        shingles = []
        size = len(doc)
        for i in range(size-k):
            shingles.append(doc[i:i+k])
        all_shingles.append(shingles)
        
    return all_shingles

# It's easy to remove spaces - simply join the document on the empty string instead of on ' '.
joined = [' '.join(document) for document in stripped_documents]
print(k_shingles_characters(joined)[0])

['t12', '120', '20 ', '0 T', ' Th', 'The', 'he ', 'e S', ' Su', 'Sup', 'upr', 'pre', 'rem', 'eme', 'me ', 'e C', ' Co', 'Cou', 'our', 'urt', 'rt ', 't J', ' Jo', 'Joh', 'ohn', 'hnn', 'nne', 'nes', 'esb', 'sbe', 'ber', 'erg', 'rg ', 'g F', ' Fr', 'Fri', 'rid', 'ida', 'day', 'ay ', 'y p', ' po', 'pos', 'ost', 'stp', 'tpo', 'pon', 'one', 'ned', 'ed ', 'd M', ' Ma', 'Mar', 'arc', 'rch', 'ch ', 'h 1', ' 14', '14 ', '4 h', ' he', 'hea', 'ear', 'ari', 'rin', 'ing', 'ng ', 'g p', ' pe', 'pet', 'eti', 'tit', 'iti', 'tio', 'ion', 'on ', 'n g', ' go', 'gov', 'ove', 'ver', 'ern', 'rnm', 'nme', 'men', 'ent', 'nt ', 't m', ' mi', 'min', 'ini', 'nis', 'ist', 'ste', 'ter', 'er ', 'r W', ' Wi', 'Win', 'inn', 'nni', 'nie', 'ie ', 'e M', ' Ma', 'Man', 'and', 'nde', 'del', 'ela', 'la ', 'a p', ' pr', 'pre', 'rev', 'eve', 'ven', 'ent', 'nt ', 't p', ' po', 'pol', 'oli', 'lic', 'ice', 'ce ', 'e r', ' re', 'rea', 'ead', 'adi', 'din', 'ing', 'ng ', 'g d', ' do', 'doc', 'ocu', 'cum', 'ume', 'men', 'ent', 'nts'

# Question 5: Even More Minhash (Document Similarity)

### Part A

### Part B